# Create prototype owl database for pyiron (proof of concept)

Aim: Demonstrate the key steps to build up a simple ontology using owlready2, apply a reasoner and store it in an owl file. I used here owlready since it appears to be presently the only tool where a reasoner is available (which is important for the ontology community). However, it would be easy to realize the same functionality with database tools such as sqlalchemy.

In [1]:
import owlready2 as owl
import pandas

In [2]:
# Make sure that the java path is correct
# from shutil import which
# which('java')

## Construct owl file fully in python and convert it into a csv file

This is useful for demonstration purposes. In the actual implementation we will start directly with the csv file that can be obtained e.g. from the pyironObject or from the ironFlow nodes.

In [3]:
onto = owl.get_ontology("file://pyiron.owl")

with onto:
    class PyObject(owl.Thing):
        comment = 'my pyiron object'
    
    class Parameter(PyObject):
        @property
        def description(self):
            if len(self.generic_parameter) > 0:
                return self.generic_parameter[0].description
            
    class InputParameter(Parameter): 
        # @property
        def consistent_output(self, additional_conditions=None):
            if additional_conditions is None:
                conditions = self.has_conditions
            else:
                conditions = additional_conditions
            if len(conditions) > 0:    
                return [p for p in self.generic_parameter[0].has_parameters if is_subset(conditions, p.has_options)]
                # return list(set.intersection(*map(set,[i.has_optional_objects for i in conditions])))
            return self.generic_parameter[0].has_parameters

    # class MandatoryInputParameter(InputParameter): pass         
        
    class OutputParameter(Parameter): pass
    class GenericParameter(Parameter): 
        description = ""
          
    class Code(Parameter): pass
        # generic_parameter=[Executable]
    # class AtomicStructure(Parameter):
    #     decription='Atomic structure. Contains at least positions and chemical species.'
    class Label(PyObject): pass    
    
    class has_conditions(Parameter >> Label): pass
    class has_transitive_conditions(Parameter >> Label): pass  # condition to fulfill to fulfill option in code 
    class has_options(Parameter >> Label): pass

    class has_transitive_objects(Label >> Parameter):
        inverse_property = has_transitive_conditions     
        
    class has_conditional_objects(Label >> Parameter):
        inverse_property = has_conditions  
        
    class has_optional_objects(Label >> Parameter):
        inverse_property = has_options    
        
    class has_symbol(GenericParameter >> str):
        class_property_type = ["some"]
        python_name = "symbols" 
        
    class has_unit(Parameter >> str):
        class_property_type = ["some"]
        python_name = "unit"        
        
    class is_in_domains (Parameter >> Label): 
        class_property_type = ["some"]
        python_name = "domain" 
        
    class domain_has_codes(Label >> Parameter):
        python_name = "has_objects" 
        inverse_property = is_in_domains   
        
    class has_generic_parameter(Parameter >> GenericParameter):
        class_property_type = ["some"]
        python_name = "generic_parameter" 
        
    class generic_parameter_has(GenericParameter >> Parameter):
        python_name = "has_parameters" 
        inverse_property = has_generic_parameter        
        
    class is_input_of (InputParameter >> Code):
        class_property_type = ["some"]
        python_name = "input_in"  
        
    class has_input (Code >> InputParameter):
        python_name = "input" 
        inverse_property = is_input_of  
        
    class is_mandatory_input_of (InputParameter >> Code):
        class_property_type = ["some"]
        python_name = "mandatory_input_in"  
        
    class has_mandatory_input (Code >> InputParameter):
        python_name = "mandatory_input" 
        inverse_property = is_mandatory_input_of          
        
    class is_output_of (OutputParameter >> Code):
        class_property_type = ["some"]
        python_name = "output_of"      # python name has to be unique (even for different class)
        
    class has_output (Code >> OutputParameter):
        python_name = "output" 
        inverse_property = is_output_of          
        
    owl.AllDisjoint([InputParameter, OutputParameter, Label])
    
    
lblAtomistic = Label(name='Atomistic')    
lblCode = Label(name='lCode')
lblDFT = Label(name='DFT')
lblMaterialProperty = Label(name='MaterialProperty')
lblPeriodicBoundaryConditions = Label(name='PeriodicBoundaryConditions')
lblUserInput = Label(name='UserInput', comment='Easy to provide input. Can be used to start a workflow')
lblBulk3DCrystal = Label(name='Bulk3dStructure', comment='Bulk 3d structure generated/needed. Has a well defined volume.')
lblAtomisticEnergyCalculator = Label(name='AtomisticEnergyCalculator', comment='Code to compute the energy of an atomic structure')


ChemicalElement = GenericParameter(name='ChemicalElement', 
                                   description='Single chemical element', 
                                   domain=[lblAtomistic, lblUserInput])
AtomicStructure = GenericParameter(name='AtomicStructure', 
                                   description='Contains all information to construct an atomic structure (molecule, crystal, etc.)', 
                                   domain=[lblAtomistic])
Executable = GenericParameter(name='Executable', description='Code that requires input and produces output', 
                                   domain=[])

# Structure
CreateStructureBulk = Code(name='CreateStructureBulk', 
                           domain=[lblAtomistic, lblCode], 
                           generic_parameter=[Executable]) 
CreateStructureBulk_element = InputParameter(name=f'{CreateStructureBulk.name}/input/element', 
                                             mandatory_input_in=[CreateStructureBulk], 
                                             generic_parameter=[ChemicalElement])  
CreateStructureBulk_structure = OutputParameter(name=f'{CreateStructureBulk.name}/output/structure', output_of=[CreateStructureBulk]
                                                , generic_parameter=[AtomicStructure], has_options=[lblBulk3DCrystal]) 

CreateSurface = Code(name='CreateSurface', 
                           domain=[lblAtomistic, lblCode], 
                           generic_parameter=[Executable]) 
CreateSurface_element = InputParameter(name=f'{CreateSurface.name}/input/element', 
                                             mandatory_input_in=[CreateSurface], 
                                             generic_parameter=[ChemicalElement])  
CreateSurface_structure = OutputParameter(name=f'{CreateSurface.name}/output/structure', output_of=[CreateSurface]
                                                , generic_parameter=[AtomicStructure], has_options=[]) 


# Murnaghan
Bulkmodulus = GenericParameter(name='Bulk_modulus', 
                               description='https://en.wikipedia.org/wiki/Bulk_modulus', 
                               symbols=['B', 'K'], 
                               unit=['MPa'], 
                               domain=[lblMaterialProperty])
Bprime = GenericParameter(name='B_prime', 
                          decription='First derivative of Bulk modulus with respect to volume', 
                          symbols=['Bprime'], 
                          unit=['1'], 
                          domain=[lblMaterialProperty])

Murnaghan = Code(name='Murnaghan', 
                 domain=[lblAtomistic, lblCode], 
                 generic_parameter=[Executable]) 
Murnaghan_Bulkmodulus = OutputParameter(name=f'{Murnaghan.name}/output/equilibrium_bulk_modulus', 
                                        output_of=[Murnaghan], 
                                        generic_parameter=[Bulkmodulus], 
                                        unit=['GPa']) 
Murnaghan_Bprime = OutputParameter(name=f'{Murnaghan.name}/output/equilibrium_b_prime', 
                                   output_of=[Murnaghan], 
                                   generic_parameter=[Bprime])                                                                                                                                                                                                                                            
Murnaghan_Ref_Job = InputParameter(name=f'{Murnaghan.name}/ref_job', 
                                   mandatory_input_in=[Murnaghan], 
                                   generic_parameter=[Executable], 
                                   has_conditions=[lblBulk3DCrystal, lblAtomisticEnergyCalculator])                                                                                                                      

# DFT
EnergyCutoff = GenericParameter(name='EnergyCutoff', 
                                description='The cutoff on the number of plane wave functions being utilized as basis functions to represent the wavefunction')

VASP = Code(name='VASP', domain=[lblAtomistic, lblCode, lblDFT], 
            has_options=[lblBulk3DCrystal, lblAtomisticEnergyCalculator], 
            generic_parameter=[Executable])        
VASP_ENCUT = InputParameter(name='ENCUT', 
                            input_in=[VASP], 
                            generic_parameter=[EnergyCutoff], 
                            unit=['eV']) 
VASP_IBRAV = InputParameter(name='IBRAV', 
                            input_in=[VASP]) 
VASP_Structure = InputParameter(name=f'{VASP.name}/input/structure', 
                                mandatory_input_in=[VASP], 
                                generic_parameter=[AtomicStructure], 
                                has_transitive_conditions=[lblBulk3DCrystal])   

VASP_ETOT = OutputParameter(name='ETOT', output_of=[VASP]) 

# LAMMPS

LAMMPS = Code(name='LAMMPS', domain=[lblAtomistic, lblCode], 
            has_options=[lblBulk3DCrystal, lblAtomisticEnergyCalculator], 
            generic_parameter=[Executable])        

LAMMPS_Structure = InputParameter(name=f'{LAMMPS.name}/input/structure', 
                                mandatory_input_in=[LAMMPS], 
                                generic_parameter=[AtomicStructure], 
                                has_transitive_conditions=[lblBulk3DCrystal])   

LAMMPS_ETOT = OutputParameter(name='ETOT', output_of=[LAMMPS]) 

owl.close_world(PyObject) 
owl.sync_reasoner_pellet(infer_property_values = True, infer_data_property_values = True, debug=0)

onto.save()

def is_subset(a, b):
    return np.all([aa in b for aa in a])

In [36]:
onto_orig = owl.get_ontology("file://pyiron.owl").load()

#### A few code examples how to access ontology

In [37]:
individuals = list(onto_orig.individuals())

In [38]:
i = individuals[0]
ii = list(i.get_properties())[0]
ii.inverse_property, list(ii.get_relations())

(pyiron.is_in_domains,
 [(pyiron.UserInput, pyiron.ChemicalElement),
  (pyiron.Atomistic, pyiron.ChemicalElement),
  (pyiron.Atomistic, pyiron.AtomicStructure),
  (pyiron.lCode, pyiron.CreateStructureBulk),
  (pyiron.Atomistic, pyiron.CreateStructureBulk),
  (pyiron.lCode, pyiron.CreateSurface),
  (pyiron.Atomistic, pyiron.CreateSurface),
  (pyiron.MaterialProperty, pyiron.Bulk_modulus),
  (pyiron.MaterialProperty, pyiron.B_prime),
  (pyiron.lCode, pyiron.Murnaghan),
  (pyiron.Atomistic, pyiron.Murnaghan),
  (pyiron.lCode, pyiron.VASP),
  (pyiron.Atomistic, pyiron.VASP),
  (pyiron.DFT, pyiron.VASP),
  (pyiron.lCode, pyiron.LAMMPS),
  (pyiron.Atomistic, pyiron.LAMMPS)])

### Extract pandas Table and csv-file from ontology

In [39]:
inverse_list = ['has_objects', 'has_transitive_objects', 'has_conditional_objects', 
               'has_optional_objects', 'has_parameters', 'output','mandatory_input', 'input']

obj_lst = [] 
individuals = list(onto_orig.individuals())
for i in individuals:
    obj_dict = {}
    # print (i.is_instance_of[0], i.name)
    obj_dict['class'] = i.is_instance_of[0].name
    obj_dict['name'] = i.name
    for p in list(i.get_properties()):
        if p.python_name in inverse_list:
            continue
        # print ("   ", p.python_name, getattr(i, p.python_name))
        new_item_lst = []
        item_lst = getattr(i, p.python_name)
        for item in item_lst:
            if hasattr(item, 'name'):
                new_item_lst.append(item.name)
            else:
                new_item_lst.append(item)
                
        obj_dict[p.python_name] = new_item_lst
    obj_lst.append(obj_dict)
df = pandas.DataFrame(obj_lst)  

sorter = ['Label', 'GenericParameter', 'Code', 'InputParameter', 'OutputParameter']
df['class'] = pandas.Categorical(df['class'], sorter)
df = df.sort_values(by='class')
df.to_csv('pyiron_onto.csv', index=False)

df

,class,name,comment,domain,generic_parameter,mandatory_input_in,has_options,output_of,symbols,unit,has_conditions,input_in,has_transitive_conditions
0,Label,Atomistic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Label,lCode,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Label,DFT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Label,MaterialProperty,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Label,PeriodicBoundaryConditions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Label,UserInput,[Easy to provide input. Can be used to start a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Label,Bulk3dStructure,[Bulk 3d structure generated/needed. Has a wel...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Label,AtomisticEnergyCalculator,[Code to compute the energy of an atomic struc...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,GenericParameter,B_prime,NaN,[MaterialProperty],NaN,NaN,NaN,NaN,[Bprime],[1],NaN,NaN,NaN
17,GenericParameter,Bulk_modulus,NaN,[MaterialProperty],NaN,NaN,NaN,NaN,"[B, K]",[MPa],NaN,NaN,NaN


## Read data from csv file

Define only classes and read the actual data from csv file. You can directly start here without running the first part.

In [8]:
onto = owl.get_ontology("file://pyiron_csv.owl")

with onto:
    class PyObject(owl.Thing):
        comment = 'my pyiron object'
    
    class Parameter(PyObject):
        @property
        def description(self):
            if len(self.generic_parameter) > 0:
                return self.generic_parameter[0].description
            
    class InputParameter(Parameter): 
        # @property
        def consistent_output(self, additional_conditions=None):
            if additional_conditions is None:
                conditions = self.has_conditions
            else:
                conditions = additional_conditions
            if len(conditions) > 0:    
                return [p for p in self.generic_parameter[0].has_parameters if is_subset(conditions, p.has_options)]
                # return list(set.intersection(*map(set,[i.has_optional_objects for i in conditions])))
            return self.generic_parameter[0].has_parameters

    # class MandatoryInputParameter(InputParameter): pass         
        
    class OutputParameter(Parameter): pass
    class GenericParameter(Parameter): 
        description = ""
          
    class Code(Parameter): pass

    class Label(PyObject): pass    
    
    class has_conditions(Parameter >> Label): pass
    class has_transitive_conditions(Parameter >> Label): pass  # condition to fulfill to fulfill option in code 
    class has_options(Parameter >> Label): pass

    class has_transitive_objects(Label >> Parameter):
        inverse_property = has_transitive_conditions     
        
    class has_conditional_objects(Label >> Parameter):
        inverse_property = has_conditions  
        
    class has_optional_objects(Label >> Parameter):
        inverse_property = has_options    
        
    class has_symbol(GenericParameter >> str):
        class_property_type = ["some"]
        python_name = "symbols" 
        
    class has_unit(Parameter >> str):
        class_property_type = ["some"]
        python_name = "unit"        
        
    class is_in_domains (Parameter >> Label): 
        class_property_type = ["some"]
        python_name = "domain" 
        
    class domain_has_codes(Label >> Parameter):
        python_name = "has_objects" 
        inverse_property = is_in_domains   
        
    class has_generic_parameter(Parameter >> GenericParameter):
        class_property_type = ["some"]
        python_name = "generic_parameter" 
        
    class generic_parameter_has(GenericParameter >> Parameter):
        python_name = "has_parameters" 
        inverse_property = has_generic_parameter        
        
    class is_input_of (InputParameter >> Code):
        class_property_type = ["some"]
        python_name = "input_in"  
        
    class has_input (Code >> InputParameter):
        python_name = "input" 
        inverse_property = is_input_of  
        
    class is_mandatory_input_of (InputParameter >> Code):
        class_property_type = ["some"]
        python_name = "mandatory_input_in"  
        
    class has_mandatory_input (Code >> InputParameter):
        python_name = "mandatory_input" 
        inverse_property = is_mandatory_input_of          
        
    class is_output_of (OutputParameter >> Code):
        class_property_type = ["some"]
        python_name = "output_of"      # python name has to be unique (even for different class)
        
    class has_output (Code >> OutputParameter):
        python_name = "output" 
        inverse_property = is_output_of          
        
    owl.AllDisjoint([InputParameter, OutputParameter, Label])

onto.save()

def is_subset(a, b):
    return np.all([aa in b for aa in a])

In [9]:
onto = owl.get_ontology("file://pyiron_csv.owl")
onto.load()

get_ontology("file://pyiron_csv.owl#")

#### Load csv files as pandas table

In [10]:
# pandas.read_csv('parameters.csv', skipinitialspace=True,  delimiter=',').iloc[0].domain

In [11]:
eval(pandas.read_csv('pyiron_onto.csv', skipinitialspace=True,  delimiter=',').iloc[8].domain)

['MaterialProperty']

In [12]:
df_para = pandas.read_csv('pyiron_onto.csv', skipinitialspace=True,  delimiter=',')
df_para

,class,name,comment,domain,generic_parameter,mandatory_input_in,has_options,output_of,symbols,unit,has_conditions,input_in,has_transitive_conditions
0,Label,Atomistic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Label,lCode,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Label,DFT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Label,MaterialProperty,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Label,PeriodicBoundaryConditions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Label,UserInput,['Easy to provide input. Can be used to start ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Label,Bulk3dStructure,['Bulk 3d structure generated/needed. Has a we...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Label,AtomisticEnergyCalculator,['Code to compute the energy of an atomic stru...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,GenericParameter,B_prime,NaN,['MaterialProperty'],NaN,NaN,NaN,NaN,['Bprime'],['1'],NaN,NaN,NaN
9,GenericParameter,Bulk_modulus,NaN,['MaterialProperty'],NaN,NaN,NaN,NaN,"['B', 'K']",['MPa'],NaN,NaN,NaN


In [13]:
import numpy as np

non_ontology_keys = ['symbols', 'unit']

def get_args(i_0, df):
    qwargs = {}
    # print ('class: ', df.iloc[i_0]['class'])  
    for key in df.keys():
        if key in ['class']:
            continue  
        val = df.iloc[i_0][key]
        if val is not np.nan:
            # print (key, val)
            if isinstance(val, str):
                val = val.strip()
                if key == 'comment':
                    val = val[2:-2]
                # print (key, val)
                if len(val) == 0:
                    continue
                elif val[0] == '[':  # list
                    val_lst = eval(val)
                    if key not in non_ontology_keys:
                        val_lst = [onto_labels[d.strip()] for d in val_lst]
                    qwargs[key] = val_lst
                else:       
                    qwargs[key] = val
                    
    return qwargs     

In [14]:
# get_args(17, df_para)

In [15]:
import pandas
import types

# df_labels = pandas.read_csv('label.csv', skipinitialspace=True)

onto_labels = owl.get_ontology("file://pyiron_labels.owl")
onto_labels.imported_ontologies.append(onto)

with onto_labels:
#     for index, row in df_labels.iterrows():
#         lbl = onto.Label(name=row['name'].strip())
#         if isinstance(row['comment'], str):
#             lbl.comment = row['comment'].strip()
        
    for index, row in df_para.iterrows():   
        if isinstance(row['class'], str):
            parent = onto[row['class']]
            # print (parent, row['name'])
            if parent is None:
                print ('Invalid class:', parent)
                continue

            qwargs = get_args(index, df_para)  
            print (row['class'], parent, qwargs)
            individuum = parent(**qwargs)  
            
    owl.close_world(PyObject) 
    owl.sync_reasoner_pellet(infer_property_values = True, infer_data_property_values = True, debug=0)                
onto_labels.save()        

Label pyiron_csv.Label {'name': 'Atomistic'}
Label pyiron_csv.Label {'name': 'lCode'}
Label pyiron_csv.Label {'name': 'DFT'}
Label pyiron_csv.Label {'name': 'MaterialProperty'}
Label pyiron_csv.Label {'name': 'PeriodicBoundaryConditions'}
Label pyiron_csv.Label {'name': 'UserInput', 'comment': 'Easy to provide input. Can be used to start a workflow'}
Label pyiron_csv.Label {'name': 'Bulk3dStructure', 'comment': 'Bulk 3d structure generated/needed. Has a well defined volume.'}
Label pyiron_csv.Label {'name': 'AtomisticEnergyCalculator', 'comment': 'Code to compute the energy of an atomic structure'}
GenericParameter pyiron_csv.GenericParameter {'name': 'B_prime', 'domain': [pyiron_labels.MaterialProperty], 'symbols': ['Bprime'], 'unit': ['1']}
GenericParameter pyiron_csv.GenericParameter {'name': 'Bulk_modulus', 'domain': [pyiron_labels.MaterialProperty], 'symbols': ['B', 'K'], 'unit': ['MPa']}
GenericParameter pyiron_csv.GenericParameter {'name': 'Executable'}
GenericParameter pyiron_c

### Read csv generated owl file

In [16]:
onto_labels = owl.get_ontology("file://pyiron_labels.owl").load()

### Explore ontology (collect/test some useful commands)

In [17]:
individuals = list(onto_labels.individuals())
individuals

[pyiron_labels.Atomistic,
 pyiron_labels.lCode,
 pyiron_labels.DFT,
 pyiron_labels.MaterialProperty,
 pyiron_labels.PeriodicBoundaryConditions,
 pyiron_labels.UserInput,
 pyiron_labels.Bulk3dStructure,
 pyiron_labels.AtomisticEnergyCalculator,
 pyiron_labels.B_prime,
 pyiron_labels.Bulk_modulus,
 pyiron_labels.Executable,
 pyiron_labels.AtomicStructure,
 pyiron_labels.ChemicalElement,
 pyiron_labels.EnergyCutoff,
 pyiron_labels.CreateStructureBulk,
 pyiron_labels.CreateSurface,
 pyiron_labels.LAMMPS,
 pyiron_labels.Murnaghan,
 pyiron_labels.VASP,
 pyiron_labels.VASP/input/structure,
 pyiron_labels.IBRAV,
 pyiron_labels.ENCUT,
 pyiron_labels.CreateSurface/input/element,
 pyiron_labels.CreateStructureBulk/input/element,
 pyiron_labels.Murnaghan/ref_job,
 pyiron_labels.LAMMPS/input/structure,
 pyiron_labels.Murnaghan/output/equilibrium_b_prime,
 pyiron_labels.Murnaghan/output/equilibrium_bulk_modulus,
 pyiron_labels.CreateSurface/output/structure,
 pyiron_labels.CreateStructureBulk/output

In [18]:
onto_labels['CreateStructureBulk'].get_properties()

{pyiron_csv.has_mandatory_input,
 pyiron_csv.has_output,
 pyiron_csv.has_generic_parameter,
 pyiron_csv.is_in_domains}

In [19]:
onto_labels.Bulk_modulus.has_parameters

[pyiron_labels.Murnaghan/output/equilibrium_bulk_modulus]

In [20]:
onto_labels.get_imported_ontologies()

[get_ontology("file://pyiron_csv.owl#")]

In [21]:
onto_labels.get_instances_of(onto.GenericParameter)

[pyiron_labels.B_prime,
 pyiron_labels.Bulk_modulus,
 pyiron_labels.Executable,
 pyiron_labels.AtomicStructure,
 pyiron_labels.ChemicalElement,
 pyiron_labels.EnergyCutoff]

In [22]:
list(onto.classes())

[pyiron_csv.PyObject,
 pyiron_csv.Parameter,
 pyiron_csv.InputParameter,
 pyiron_csv.OutputParameter,
 pyiron_csv.GenericParameter,
 pyiron_csv.Code,
 pyiron_csv.Label]

In [23]:
list(onto.data_properties())

[pyiron_csv.has_symbol, pyiron_csv.has_unit]

In [24]:
onto.Parameter.descendants()

{pyiron_csv.Code,
 pyiron_csv.GenericParameter,
 pyiron_csv.InputParameter,
 pyiron_csv.OutputParameter,
 pyiron_csv.Parameter}

In [25]:
# onto.Parameter.get_class_properties()

In [26]:
properties = list(onto.object_properties())
properties

[pyiron_csv.has_conditions,
 pyiron_csv.has_transitive_conditions,
 pyiron_csv.has_options,
 pyiron_csv.has_transitive_objects,
 pyiron_csv.has_conditional_objects,
 pyiron_csv.has_optional_objects,
 pyiron_csv.is_in_domains,
 pyiron_csv.domain_has_codes,
 pyiron_csv.has_generic_parameter,
 pyiron_csv.generic_parameter_has,
 pyiron_csv.is_input_of,
 pyiron_csv.has_input,
 pyiron_csv.is_mandatory_input_of,
 pyiron_csv.has_mandatory_input,
 pyiron_csv.is_output_of,
 pyiron_csv.has_output]

In [27]:
onto_labels.DFT.is_a[0].instances()

[pyiron_labels.Atomistic,
 pyiron_labels.lCode,
 pyiron_labels.DFT,
 pyiron_labels.MaterialProperty,
 pyiron_labels.PeriodicBoundaryConditions,
 pyiron_labels.UserInput,
 pyiron_labels.Bulk3dStructure,
 pyiron_labels.AtomisticEnergyCalculator]

In [28]:
onto_labels.Bulk3dStructure

pyiron_labels.Bulk3dStructure

In [29]:
onto_labels.get_imported_ontologies()

[get_ontology("file://pyiron_csv.owl#")]

In [30]:
list(onto.classes())

[pyiron_csv.PyObject,
 pyiron_csv.Parameter,
 pyiron_csv.InputParameter,
 pyiron_csv.OutputParameter,
 pyiron_csv.GenericParameter,
 pyiron_csv.Code,
 pyiron_csv.Label]

In [31]:
onto.Label.instances()[-2].get_iri()

'file://pyiron_labels.owl#Bulk3dStructure'

In [32]:
onto.base_iri

'file://pyiron_csv.owl#'

In [33]:
onto.Label.instances()

[pyiron_labels.Atomistic,
 pyiron_labels.lCode,
 pyiron_labels.DFT,
 pyiron_labels.MaterialProperty,
 pyiron_labels.PeriodicBoundaryConditions,
 pyiron_labels.UserInput,
 pyiron_labels.Bulk3dStructure,
 pyiron_labels.AtomisticEnergyCalculator]

In [34]:
onto_labels.AtomisticEnergyCalculator.is_a[0].instances()

[pyiron_labels.Atomistic,
 pyiron_labels.lCode,
 pyiron_labels.DFT,
 pyiron_labels.MaterialProperty,
 pyiron_labels.PeriodicBoundaryConditions,
 pyiron_labels.UserInput,
 pyiron_labels.Bulk3dStructure,
 pyiron_labels.AtomisticEnergyCalculator]